### Da guardare ?
- tratto gli algoritmi di ottimizzaz come con ROOT ie a sentimento (learning_rate? arriva dla teoria!! sia lode)
- batch norm (qualitativam ok)
- pké si parli di API -> pké sto facendo parlare tensorflow e keras?

https://keras.io/api/ c'è tutta la documentazione con esempi credo!!

probab c'è anche 1 bella documentazione di tensorflow (con tnt di video?)
https://www.tensorflow.org/api_docs/python/tf e https://www.tensorflow.org/guide

video del mio bro coreano: https://www.youtube.com/@Deeplearningai/playlists

tutorial del mio altro bruh: https://www.youtube.com/playlist?list=PLhhyoLH6IjfxVOdVC1P1L5z5azs0XjMsb

reference per python (?): https://realpython.com/

MIT material (videos code and slides): http://introtodeeplearning.com/2023/index.html

# Basics

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # per sbarazzarsi dei messaggi di info di tf

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# it may be necessary to import matplotlib.pyplot

# to avoid GPU errors
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# Creiamo dei tensori
x = tf.constant([[2,3,4],[5,6,7]], shape=(2,3), dtype=tf.float32)

y = tf.eye(4) # produce la matrice identità dla dimens desiderata

# posso generarli da distribuzioni di probab e usando tf.range(start,limit,delta)
x = tf.cast(x,dtype = tf.float32) # per convertire tra i tipi

x = tf.reshape(x, (6,))
indices = tf.constant([2,4])
x_ind = tf.gather(x,indices) # con x vettore e indices vettore di indici da estrarre
print(x_ind)
print(tf.shape(x))
x = tf.reshape(x,(2,3))
print(x)

tf.Tensor([4. 6.], shape=(2,), dtype=float32)
tf.Tensor([6], shape=(1,), dtype=int32)
tf.Tensor(
[[2. 3. 4.]
 [5. 6. 7.]], shape=(2, 3), dtype=float32)


le operaz le fa già da sé element-wise
`z = tf.tensordot(x,y,axes=1)` fa il prodotto scalare (ma è in grado di fare molto di +), in sost credo che axis mi dica su qle dimensione sto agendo
`tf.matmul(a,b) = a @ b` per le moltipl tra matrici

l'indexing funziona come il solito python, `x[2:3]` con 2 incluso e 3 escluso, `:` da solo significa tutti e posso ancora aggiungere un altro colon `:step`

e qsti giochini sugli indici si estendono chiaramente a ogg di dimensione maggiore

# Object Oriented Python
classes are data structures (ie blueprints) and the objects are their instances

In [ ]:
class ClExample(): # this is the convention for class names
# between the () goes the parent classes in case of inheritance
# qua (ie fuori da __init__) dichiaro e assegno le class variables (qle =i per tutte le instances)

  def __init__(self, attribute1, attribute2): # stands for initialize (è il costruttore direi), self as first attribute is mandatory
  # qua sto contemporaneam dichiarando gli attributi e utilizzando le corrisp funzioni set
  # metodi dla classe (ie le funzioni precipue dla classe)
  # anche .__init__ è 1 method (qndo crei 1 instance stai in sostanza chiamando il metodo costruttore ie ClExample.__init__(param))

  # super(nel caso di 1 gerarchia + complicato devo metterci i riferimenti per chiamare il costruttore giusto).__init__(qua ci vanno gli argom richiesti per inizializz l`ogg parente (senza self neh))
    self.attr1 = attribute1
    self.attr2 = attribute2

  def __str__(self):
    return 'qlo che vuoi compaia qnd printi 1 instance'
# tutti i methods che intendi def devono avere come 1o arg self


subclasses per blocchi di layers:
- eredito da layers.Layer (come costruttore `super(nome_del_blocco, self).__init__()` pké tnt il costrutt di layers nn richiede argom)
- come attributi metto i layer elementari che userò
- 1 metodo `call` (è sicuram 1 metodo dla classe parente che sto sovrascrivendo… come puoi ben capire ce ne sono altri… es build) dove costruisco il blocco nel modo functional ma senza gli `(x)` al fondo, e qsto metodo returna l'ultimo strato/il suo output?
(dentro il `call` posso chiamare print e lui mi stampa qlo che mi interessa mentre allena, ie ogni volta che viene chiamato qlo strato)
- se voglio definire 1 classe di modelli (al posto che definirlo in 1 funz posso farlo in 1 classe) la parent class dev'essere `keras.Model`, per il resto è tutto = (ma d'altronde i modelli nn sono forse blocchi di layer con qlche metodo in +…)

# Data and basic model
overgap between training e test accuracy is a symptom of **overfitting** -> solution: regularization

if the distribution of the inputs changes it maybe needed to retrain the model (magari la legge e la stsa ma i dati sono finiti fuori dal campo di misura precedente e dnq nn siamo in grado di dare previsioni accurate presumibilmente) this is called *covariance shift* (->batch normalization)

altra idea è: gioca con i param per capire trovare compromessi (abbast epoch ma nn troppe, l'optimaizer che lavora meglio…)

NN tutti i param sono trainable -> e certe cose (ad es la BatchNorm) si comportano in modo diverso a 2a se sono in training o testing -> è il caso di settare `training=True/False` nei param del model

In [ ]:
# qlo che esce da qua (e con cui lavoriamo dopo) sono array numpy, ma ci va bene = (converte da solo)
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# dobbiamo appiattire il tutto
x_train = x_train.reshape(-1, 28*28).astype("float32")/255.0 # 28*28 sono i pixel dl'immagine, -1 serve a nn toccare la 1a dimensione
# / 255.0 è normalizzaz a 1, la macchina ci mette - (?)
x_test = x_test.reshape(-1, 28*28).astype("float32")/255.0
print(tf.shape(x_train))

11490434/11490434 [==============================] - 0s 0us/step
tf.Tensor([60000   784], shape=(2,), dtype=int32)


In [ ]:
# SEQUENTIAL API (1 input maps to 1 output)
model = keras.Sequential(
    [
        keras.Input(shape=(28*28)), # qsto è in realtà il primo strato (che ha il compito di accogliere gli input), qli dopo sono gli hidden layers
        layers.Dense(512, activation = 'relu'), # Dense significa che tutti in nodi sono collegati con tutti
        layers.Dense(256, activation = 'relu'),
        layers.Dense(10) # qsto è l'output con le 10 possibili cifre
    ]
)
print(model.summary())
# è 1 debugging tool (come sempre d'altronde i print), se uso model.add() per aggiungere i layers 1 alla volta posso printare il summary dopo ogni strato

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dense_13 (Dense)            (None, 256)               131328    
                                                                 
 dense_14 (Dense)            (None, 10)                2570      
                                                                 
Total params: 535818 (2.04 MB)
Trainable params: 535818 (2.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# tell keras HOW to train the model
model.compile(
    # set softmax activation for the output layer through the loss function (with the from_logits argument), otherwise just set it in the model
    # Sparse is needed because the labels are just numbers (otherwise avremmo bisogno di 1 hot encodings)
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"] # what we keep track of during the training
)

# h specifichiamo gli aspetti pratici del training
model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2) # con verbose=2 printa dopo ogni epoch

# HYPERPARAMETERS (?): learning rate, lambd
# 1 epoch = 1 passaggio dl'intero dataset per il training algorithm
# batch size = dopo qnti sample aggiornare i param del modello/verificare i risultati(?)
model.evaluate(x_test, y_test, batch_size=32, verbose=2) # qua epoch = 1 (mica ci interessa controllare + volte…)

Epoch 1/5
1875/1875 - 16s - loss: 0.1307 - accuracy: 0.9632 - 16s/epoch - 8ms/step
Epoch 2/5
1875/1875 - 17s - loss: 0.0544 - accuracy: 0.9829 - 17s/epoch - 9ms/step
Epoch 3/5
1875/1875 - 15s - loss: 0.0382 - accuracy: 0.9877 - 15s/epoch - 8ms/step
Epoch 4/5
1875/1875 - 16s - loss: 0.0301 - accuracy: 0.9902 - 16s/epoch - 9ms/step
Epoch 5/5
1875/1875 - 17s - loss: 0.0252 - accuracy: 0.9921 - 17s/epoch - 9ms/step
313/313 - 1s - loss: 0.0808 - accuracy: 0.9787 - 851ms/epoch - 3ms/step


[0.08082181960344315, 0.9786999821662903]

In [ ]:
#import sys
#sys.exit()
# serve per terminare il programma/script in a graceful way :)
# it raises a SystemExit exception, signaling an intention to exit the interpreter
# ma sì serve per nn fargli tirare dritto ad eseguire qlo che viene dopo (è 1 va a caghè)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 512)               401920    
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 10)                2570      
                                                                 
Total params: 535818 (2.04 MB)
Trainable params: 535818 (2.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# FUNCTIONAL API
inputs = keras.Input(shape=(28*28))
x = layers.Dense(512, activation='relu', name = 'first_layer')(inputs)
x = layers.Dense(256, activation='relu', name = 'second_layer')(x) # nn so bene come interpretare qsta roba pké h stiamo definendo (e sovrascrivendo x) ma dopo dovremo usarlo…
outputs = layers.Dense(10, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs) # qua stiamo raccattando tutto {}

# il vantaggio dla functional è che posso biforcare alla fine: outputs1 = …(x) e outputs2 = …(x), in keras.Model(…, outputs=[outputs1,outputs2])
# qua sotto in compile posso avere loss function e metriche diverse (uso 1 lista come 1a)

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False), # pké abbiamo già specificato l'attivazione degli outputs, clipnorm = 1 evita exploding gradient problems
        optimizer=keras.optimizers.Adam(learning_rate=0.001), # learning rate
    metrics=["accuracy"]
)

model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/5
1875/1875 - 12s - loss: 1.0014 - accuracy: 0.7725 - 12s/epoch - 7ms/step
Epoch 2/5
1875/1875 - 12s - loss: 0.4465 - accuracy: 0.8857 - 12s/epoch - 6ms/step
Epoch 3/5
1875/1875 - 12s - loss: 0.3659 - accuracy: 0.9007 - 12s/epoch - 6ms/step
Epoch 4/5
1875/1875 - 11s - loss: 0.3279 - accuracy: 0.9096 - 11s/epoch - 6ms/step
Epoch 5/5
1875/1875 - 14s - loss: 0.3037 - accuracy: 0.9155 - 14s/epoch - 7ms/step
313/313 - 1s - loss: 0.2826 - accuracy: 0.9212 - 819ms/epoch - 3ms/step


[0.28261861205101013, 0.9211999773979187]

1 volta che ho 1 model posso lavorarci sopra…
- accedere ai layer: `model.layers[index]` con le solite convenzioni, `model.get_layer(name='layer_name')` (dove il nome lo setto qnd definisco il layer)
- estrarre input e output: `layer.inputs` e `layer.output` (dove chn layer intendo che ne hai richiamato 1 in qlche modo)
… e ad esempio usare il suo output per allenare qlche layer a valle (per farlo devo `model.trainable = False` sul modello che voglio che tenere fisso, altrimenti posso agire sui singoli layers con `layer.trainable = False`, dove con layer intendo che ne hai richiamato uno)

In [ ]:
model = keras.Model(inputs=model.inputs,
                    outputs=[model.layers[-2].output]) # con il -2 stiamo andando a vedere l'ultimo hidden layer
# che cosa sta succedendo qua? (per debuggare tipicam) sto definendo 1 nuovo modello = al mio (e con gli stsi valori dei param direi) ma a cui tolgo strati dal fondo
# poi per andare a vedere nn devo far altro che far girare qsto nuovo modello su dle x a piacere

# accedere ai layers: model.layers[index] (dove il sistema di indici funziona come per i vettori) il .output mi sfugge
#                     model.get_layer(name="layer_name")

feature = model.predict(x_train) # puoi far girare il modello su qlo che ti pare
print(tf.shape(feature))

1875/1875 [==============================] - 4s 2ms/step
tf.Tensor([60000   512], shape=(2,), dtype=int32)


Posso lavorare indistintam con array di ogg ie nel mio definire `model` posso tranquillam mettere `output=[layer.output for layer in model.layers]` e ottenere 1 array/tensor di modelli!

# Convolutional Neural Networks

al solito con la convoluzione si passa una maschera/(filtro/kernel) (dove la maschera è deputata a riconoscere qlche tipo di pattern)/sistema di pesi (che poi sono i param da tweakare con l'allenamento, ie è il training che insegna al modello i pattern da riconoscere!) e lo si fa scorrere (puoi chiaram farla scorrere in modi fantasiosi es a passi di 2 o che, in qsto modo cambia la dimensione dl'output) prendendo i prodotti element-wise e poi sommando tutto, l'output è dimensionalm + piccolo dl'input

- **padding**: aggiungere una cornice per nn perdere troppo info dai bordi (la maschera ci passa sopra poco rispetto a celle/.i + centrali) -> permette di avere output e input con stsa dim (ie padding='same' otherwise padding='valid' se NN voglio padding)

1 dle dimensioni dl'input può essere associata ai **channel** (tipicam RGB), per convenz si chiede che il filtro abbia lo stso numero di channel (ie che il filtro nn si muova tra i channel, che ha molto senso se intendiamo che i channel portino info distinte) -> dnq l'output è piatto (ie perdiamo totalm/si annulla la dimensione a cui corrisp i channel)

in 1 passaggio possiamo applicare 2 maschere e ottenere 1 output nn piatto (ie stiamo cercando di riconoscere 2 pattern/features in contemporanea -> qsti saranno i channel dl'output! ie 1 pallozzi che rappresentano i neuroni! ie 1 pallozzo <-> 1 channel)

- **max pooling** è 1 step di pura computazione (ie nn ci sono parametri) e semplicem funziona bene. Faccio correre 1 maschera (pool) che ha il compito di estrarre il numero maggiore, nulla + (l'idea è che se in qla regione ho trovato il pattern allora c'è almeno 1 numero grande, dnq se trovare il pattern è il mio obiettivo qsto chiaramente elimina le info nn interessanti riducendo le dimensioni dl'output dl layer)

`layers.Conv2D(num di channel dl'output ie num di kernels, dim del kernel, padding='', activation)`

`layers.MaxPooling2D(pool_size= dim dla maschera)`

ecco che h è utile il `print(model.summary())` per tenere traccia dle varie dimensioni e parametri

qsto genere di layer qua agisce su dati organizzati sotto forma di tensori, se voglio poi passare a 1 strato Dense (ad es) devo appiattire tutto con `layers.Flatten()`


In [ ]:
x_train = tf.reshape(x_train,(-1,28,28))
x_test = x_test.reshape(-1,28,28) # ok quick refresh: applicare 1 metodo (ie le cose con il .) nn agisce modificando l'ogg è indisp riassegnare
print(tf.shape(x_train))
print(tf.shape(x_test))

tf.Tensor([60000    28    28], shape=(3,), dtype=int32)
tf.Tensor([10000    28    28], shape=(3,), dtype=int32)


In [ ]:
def convnet_model():
  inputs = keras.Input(shape=(28,28,1)) # qsto 1 finale è indisp *******>>>> riguardami ;)
  x = layers.Conv2D(32,(3,3),padding='valid',activation='relu')(inputs)
  x = layers.MaxPooling2D(pool_size=(2,2))(x)
  x = layers.Conv2D(16,3)(x)
  x = layers.BatchNormalization()(x)
#  x = keras.activations.relu(x) # nn è 1 layer, stiamo settando 1 opzione del layer e va fatto dopo BatchNorm
  x = tf.nn.relu(x) # credo sia analogo al metodo sopra
  x = layers.Flatten()(x) # qsto invece sì che è 1 layer
  x = layers.Dense(16,activation='relu')(x)
  outputs = layers.Dense(10)(x) # l'activation di qsti sarà settata dopo

  model = keras.Model(inputs=inputs, outputs=outputs)
  return model

model = convnet_model()

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), # pké abbiamo già specificato l'attivazione degli outputs
        optimizer=keras.optimizers.Adam(learning_rate=0.001), # learning rate
    metrics=["accuracy"]
)

model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)


Epoch 1/5
1875/1875 - 19s - loss: 0.1729 - accuracy: 0.9466 - 19s/epoch - 10ms/step
Epoch 2/5
1875/1875 - 18s - loss: 0.0508 - accuracy: 0.9842 - 18s/epoch - 10ms/step
Epoch 3/5
1875/1875 - 18s - loss: 0.0385 - accuracy: 0.9877 - 18s/epoch - 10ms/step
Epoch 4/5
1875/1875 - 18s - loss: 0.0300 - accuracy: 0.9902 - 18s/epoch - 9ms/step
Epoch 5/5
1875/1875 - 18s - loss: 0.0243 - accuracy: 0.9920 - 18s/epoch - 10ms/step
313/313 - 1s - loss: 0.0331 - accuracy: 0.9885 - 1s/epoch - 4ms/step


[0.03313692659139633, 0.9884999990463257]

# Overfitting & Regularization
obb: shrinkare i weights (ie diminuire i gdl o perlomeno nn lasciarli liberi del tutto)
anche se nn hai overfitting ha cmq senso provare a condire il modello con 1 po' di regularization

 high variance cases leadsto very nn linear decision boundaries (pké arrivo a dei boundaries sartoriali sul training sample ma se la popolazione è sul serio con 1 high variance sicuramente nn sono riuscito ad identificare gli effettivi boundaries pké facilmente ci sono molte variazioni nn previste dai dati di allenam… la vera soluz per casi di high variance è radunare 1 training set + grande e nn ridurre i gdl ma si fa ql che si può) -> pochi dati? introduci del dropout se no overfitti

(anche la **BatchNorm** agisce 1 po' da regularization a qnt pare, e si comporta in modo diverso se sto allenando la rete o la sto testando?)
- **L2** (regularization si riferisce alla norma euclidea/di Frobenius dla matrice dei pesi) e l'idea è qla di aggiungere 1 termine (con qsta norma e pesato da 1 param) alla loss function che diminuisca gli incrementi nei parametri (dandogli - modo di aggiustarsi) -> also referred as weight decay
`kernel_regularizer=regularizers.l2(param)` va aggiunto in tutti i layer dove vuoi che avvenga (immagino la sintassi sia diversa se nn si tratta di 1 convolutional layer ma ci siamo intesi)

- **Dropout** ad ogni iteraz ti faccio sparire 1 fraz fissata (hyperparam) di unità da 1 livello. Le unità del liv successivo allenano i pesi con meno efficacia pké nn possono fare affidam su ness1 di loro (dandogli troppo importanza) -> nn si riesce a sfruttare bene qsti pesi (e noi vogliamo diminuire i gdl qndi ottimo!)
`x = layers.Dropout(perc di link da buttare)(x)` nla  functional API -> ci vogliono + epochs per allenam decentem (fa + fatica poverina…)

Data augmentation is great to avoid overfitting!! the model isn't able to memorize the training set because it's enormous and it changes! so the idea is not to make this augmentation static but to randomize it
(this augmentation proceeds in parallel during the training, and you may want to make in part of your model (does it have any sense if you then want to serialize it?))

# Save the model & Datasets

qsta roba è molto comoda per spezzare il training e riprenderlo più tardi
(devo dare i nomi ai pesi se mi sono costruito i layer custom, fai occhio)
- save & load model weights:
`model.save_weights('path')` and `model.load_weights('path')` (e nn devi assegnare qsta roba ie nn devi fare `model = model.load…`) tra qnd salvi e carichi devi utilizzare la stsa implementaz (sequential/functional) altrim ti fa 1 pernacchia

- save & load entire model (serialization): saves weights, model architecture (dnq qlo che esporto può avere vita indip dal codice in cui nasce il modello), training config (the parameters of `model.compile()` and what they have learned, es l'optimizer ha dei parametri interni che si aggiornano?)
`model.save('path')` (potrebbe comparirti 1 errore ma nn dovrebbe essere 1 pb) and `model = keras.model.load_model('path')` hai tutto pronto ie devi solo fittare

Per utilizzare 1 modello già allenato di keras `keras.applicantions.nome_del_pretrained_model()`

# Callbacks
`keras.callbacks.nome_del_callback_che_ti_interessa` -> guarda la documentazione!

posso agire sul comportam del modello nle fasi di training o evaluation (ad es posso scegliere di stoppare e salvare tutto qndo raggiungo 1 certa accuracy)

devo scegliere la callback che mi interessa settarne i param e poi darla in argom al `model.compile` o `model.fit` -> si possono avere + callbacks qndi mi aspetto che il parametro callback in qsti 2 metodi accetti 1 lista

se voglio scrivermi io 1 callback definisco 1 classe che eredita da `keras.callbacks.Callback` (e ho già dei bei methods predefiniti che permettono di scegliere il momento in cui voglio intervenire)

# Custom things
to act on fit and compile methods you have to keep track of (ie record) the operations in order to compute the gradients for the backprop

se voglio allenare il model in qlche modo particolare posso agire su `model.fit` creando 1 subclass di `keras.Model` e cambiando i methods relativi all'allenamento (ie ridef il method `train_step` che è sicuram usato under the hood da `model.fit`), altrim mi scrivo a mano il training loop e nn uso il metodo `.fit`

In [ ]:
x = tf.constant(3) # sarà poi la valutazione
with tf.GradientTape() as tape: # qsto statement serve qndo lavoro con unmanaged resources
  # (qlo che sto scrivendo è praticam 1 mini file! che dopo eseguirò qnd chiamerò gradient!)
  # operations you want to record (write them in symbols, as in Scipy I think)
  # training variables are automatically recorded
  # you can manually record a tensor object if you "watch" it
  # pké lui si ricordi/registri è suffic che 1 sola variab sia watchata
  tape.watch(x)
  y = x * x
tape.gradient(y,x) # derivo risp a x e valuto (nota l'indentazione… siamo fuori dal filmino/tape e lo stiamo usando)
# appena eseguo .gradient() si dimentica di tutto e butta il tape (se ne ho ancora bisogno: .GradientTape(persistent=True))